In [62]:
import cv2
import numpy as np
from tqdm.notebook import tqdm
from time import time
import os

LENGTH_TH = 0.15
AREA_TH = 1000

## Описание

Решение простое - будем искать на картинке квадратики с двумя и более вложениями, которые сами ни во что не вложены.
Для этого предобработаем картинку устранив некоторую часть шумов и оставим только границы с помощью оператора Кэнни. Затем отсечем все контуры, после чего отфильтруем из них те, которые подходят под наши требования.

In [63]:
def count_children(hierarchy, parent, inner=False):
    if parent == -1:
        return 0
    elif not inner:
        return count_children(hierarchy, hierarchy[parent][2], True)
    return count_children(hierarchy, hierarchy[parent][0], True) + count_children(hierarchy, hierarchy[parent][2], True) + 1

def has_square_parent(hierarchy, squares, parent):
    if hierarchy[parent][3] == -1:
        return False
    if hierarchy[parent][3] in squares:
        return True
    return has_square_parent(hierarchy, squares, hierarchy[parent][3])

def detect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(gray, 30, 200)

    contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    squares = []
    square_indices = []
    
    for i, contour in enumerate(contours):
        length = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        approx = cv2.approxPolyDP(contour, 0.03 * length, True)

        if len(approx) == 4:
            childrens = count_children(hierarchy[0], i)
            has_parent = has_square_parent(hierarchy[0], square_indices, i)
            
            if area > AREA_TH and np.abs(((length / 4) ** 2) / area - 1) < LENGTH_TH and childrens > 1 and not has_parent:
                squares.append(approx)
                square_indices.append(i)

    cv2.drawContours(image, squares, -1, (0, 0, 255), 5)

    return image

In [64]:
start = time()
for i in tqdm(["1", "2", "3"]):
    for image_name in tqdm(os.listdir("TestSet" + i)):
        image = cv2.imread("TestSet" + i + '/' + image_name)
        output = detect(image)
        cv2.imwrite("TestSet" + i + 'Result/' + image_name, output)
print(time() - start)



71.1734766960144


## Вывод
Время работы - 0.28 сек/картинка

Метрики:

| Dataset     | Precision  | Recall |
| :----:      |   :----:   | :----: |
| TestSet1    | 0.87       | 0.86   |
| TestSet2    | 0.87       | 0.78   |
| TestSet3    | 0.47       | 0.69   |

Просадку в точности можно объяснить несовершенством поиска квадратов, как таковых, поскольку на ряде картинок есть квадраты, не удовлетворявшие условиям отбора, но при этом выделенные детектором